In [2]:
import sys
import signal

# Windows does not have SIGKILL
if sys.platform == "win32":
    signal.SIGKILL = signal.SIGTERM  # fallback

In [3]:
import nemo.collections.asr as nemo_asr
import torch

# Select processing device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load Typhoon ASR Real-Time model
print("Loading Typhoon ASR Real-Time...")
asr_model = nemo_asr.models.ASRModel.from_pretrained(
    model_name="scb10x/typhoon-asr-realtime",
    map_location=device
)

Using device: cpu
Loading Typhoon ASR Real-Time...


[NeMo W 2025-11-18 16:27:40 nemo_logging:393] d:\Git\thai_voice_transcription\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TDGE0049\.cache\huggingface\hub\models--scb10x--typhoon-asr-realtime. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
    To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
      warnings.warn(message)
    


[NeMo I 2025-11-18 16:27:42 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 2048 tokens


[NeMo W 2025-11-18 16:27:44 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/workspace/warit/nemo-asr/stt_th_conformer_transducer_large/prepare_data/typhoon_cleanser/20250814/Split_gg/train_data_typhoon_asr_realtime.jsonl
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2025-11-18 16:27:44 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation d

[NeMo I 2025-11-18 16:27:44 nemo_logging:381] PADDING: 0


...

[NeMo I 2025-11-18 16:27:45 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-11-18 16:27:45 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-18 16:27:45 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-18 16:27:45 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-18 16:27:45 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-18 16:27:46 nemo_logging:381] Model EncDecRNNTBPEModel was successfully restored from C:\Users\TDGE0049\.cache\huggingface\hub\models--scb10x--typhoon-asr-realtime\snapshots\a14b79d50c788dbdfe559c8a28a9b90153cf3865\typhoon-asr-realtime.nemo.


In [ ]:
import sounddevice as sd
import numpy as np

SAMPLE_RATE = 16000
BLOCK_SIZE = 1024  # samples per callback

def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    audio_block = indata[:, 0]  # ใช้ channel แรก
    transcription = asr_model.transcribe(audio_block)
    print(">>", transcription)

print("🎤 Start capturing (Ctrl+C to stop)")
with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype="float32",
                    blocksize=BLOCK_SIZE, callback=audio_callback):
    while True:
        sd.sleep(100)

🎤 Start capturing (Ctrl+C to stop)
Audio block: [ 0.0000000e+00  0.0000000e+00 -3.0517578e-05  0.0000000e+00
  0.0000000e+00]
Audio block: [0. 0. 0. 0. 0.]
Audio block: [-0.00305176 -0.00296021 -0.00314331 -0.00323486 -0.0032959 ]
Audio block: [-0.01605225 -0.01608276 -0.01617432 -0.01617432 -0.01623535]
Audio block: [-0.01312256 -0.01306152 -0.01306152 -0.01303101 -0.01296997]
Audio block: [-0.00753784 -0.00756836 -0.00762939 -0.00765991 -0.00765991]
Audio block: [-0.00592041 -0.00595093 -0.00592041 -0.00595093 -0.00588989]
Audio block: [-0.00393677 -0.00390625 -0.00396729 -0.0039978  -0.00384521]
Audio block: [0.00137329 0.00131226 0.00125122 0.0012207  0.00125122]
Audio block: [0.00442505 0.00445557 0.00445557 0.00454712 0.00445557]
Audio block: [0.00460815 0.00463867 0.00457764 0.00466919 0.00469971]
Audio block: [0.0050354  0.00509644 0.00515747 0.00521851 0.00515747]
Audio block: [0.00506592 0.00509644 0.0050354  0.00506592 0.00491333]
Audio block: [0.00439453 0.00439453 0.004455

KeyboardInterrupt: 

In [6]:
help(asr_model.transcribe)

Help on method transcribe in module nemo.collections.asr.models.rnnt_models:

transcribe(audio: Union[str, List[str], numpy.ndarray, torch.utils.data.dataloader.DataLoader], batch_size: int = 4, return_hypotheses: bool = False, partial_hypothesis: Optional[List[ForwardRef('Hypothesis')]] = None, num_workers: int = 0, channel_selector: Union[int, Iterable[int], str, NoneType] = None, augmentor: omegaconf.dictconfig.DictConfig = None, verbose: bool = True, override_config: Optional[nemo.collections.asr.parts.mixins.transcription.TranscribeConfig] = None) -> Union[List[str], List[ForwardRef('Hypothesis')], Tuple[List[str]], Tuple[List[ForwardRef('Hypothesis')]]] method of nemo.collections.asr.models.rnnt_bpe_models.EncDecRNNTBPEModel instance
    Uses greedy decoding to transcribe audio files. Use this method for debugging and prototyping.

    Args:
        audio: (a single or list) of paths to audio files or a np.ndarray/tensor audio array or path to a manifest file.
            Can als

In [8]:
# import os
# import sys
# import time
# from pathlib import Path
# import nemo.collections.asr as nemo_asr
# import torch
# import librosa
# import soundfile as sf

# def prepare_audio(input_path, target_sr=16000):
#     input_path = Path(input_path)
#     if not input_path.exists():
#         raise FileNotFoundError(f"File not found: {input_path}")

#     supported_formats = ['.wav', '.mp3', '.m4a', '.flac', '.ogg', '.aac', '.webm']
#     if input_path.suffix.lower() not in supported_formats:
#         raise ValueError(f"Unsupported format: {input_path.suffix}. Supported formats are: {supported_formats}")

#     processed_path = f"processed_{input_path.stem}.wav"

#     y, sr = librosa.load(str(input_path), sr=None)
#     if y is None:
#         raise IOError("Failed to load audio file.")

#     duration = len(y) / sr

#     if sr != target_sr:
#         y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

#     y = y / (max(abs(y)) + 1e-8)
#     sf.write(processed_path, y, target_sr)
    
#     return processed_path, duration

# processed_path, duration = prepare_audio(input_path='audio/recorded_audio.wav')

In [11]:
import nemo.collections.asr as nemo_asr
import torch
import sounddevice as sd
import numpy as np

# # เลือก device
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(f"Using device: {device}")

# # โหลด model
# asr_model = nemo_asr.models.ASRModel.from_pretrained(
#     model_name="scb10x/typhoon-asr-realtime",
#     map_location=device
# )

# ตั้งค่าการบันทึกเสียง
sample_rate = 16000  # Typhoon model ใช้ 16kHz
block_duration = 1.0  # 1 วินาทีต่อ batch
block_size = int(sample_rate * block_duration)

print("เริ่มฟังเสียงจากไมโครโฟน... กด Ctrl+C เพื่อหยุด")

def callback(indata, frames, time, status):
    if status:
        print(status)
    audio_data = indata[:, 0]  # ใช้ channel แรก
    audio_tensor = torch.tensor(audio_data, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Transcribe
    with torch.no_grad():
        transcription = asr_model.transcribe(audio_tensor)
        print(">>", transcription)

# เริ่ม stream
with sd.InputStream(channels=1, samplerate=sample_rate, blocksize=block_size, callback=callback):
    while True:
        pass


เริ่มฟังเสียงจากไมโครโฟน... กด Ctrl+C เพื่อหยุด


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]
Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x0000024A4239E980>:
Traceback (most recent call last):
  File "d:\Git\thai_voice_transcription\.venv\Lib\site-packages\sounddevice.py", line 857, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Git\thai_voice_transcription\.venv\Lib\site-packages\sounddevice.py", line 2762, in _wrap_callback
    callback(*args)
  File "C:\Users\TDGE0049\AppData\Local\Temp\ipykernel_23152\3708563793.py", line 31, in callback
  File "d:\Git\thai_voice_transcription\.venv\Lib\site-packages\torch\utils\_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "d:\Git\thai_voice_transcription\.venv\Lib\site-packages\nemo\collections\asr\models\rnnt_models.py", line 280, in transcribe
    return sup

KeyboardInterrupt: 